# Validate ICESAT2 biomass map based on Norwegian forest resource map (SR16) created from lidar and ALS reference maps from NEON in North America
The Norwegian ALS derived AGB maps are available from https://kart8.nibio.no/nedlasting/dashboard

## Parameters to change

In [1]:
# name of directory where new tiles are
dir_output_laura= "/projects/shared-buckets/lduncanson/norway_test_data/"
dir_name="combo_night_solar" # this is the only part that one should modify to run on every different iteration

# folder where tile indexes are exported (not sure why I was not able to export in my public bucket anymore)
output_dir_tiles="/projects/my-public-bucket/norway_test_data/tiles/"
dir.create(output_dir_tiles, showWarnings=F)

# Here is to define the band number in the ICESAT-2 BIOMASS product that corresponds to the AGB
AGB_band=1 # is it really number 1?


## Load libraries

In [6]:
#install.packages('terra')
#install.packages('rgeos')
#install.packages('leaflet')
#install.packages('sp')
#install.packages('ggplot2')
library(raster)
library(terra)
library(rgeos)
library(leaflet)
library(sp)
library(gdalUtils)
library(rgdal)
library(ggplot2)
library(dplyr)

## Load reference SR16 data (native resolution 16 m)

In [7]:
AGB_data= list.files(pattern="AGB*", full=T)
AGB_data_IDs= as.numeric(gsub("[^\\d]+", "", AGB_data, perl=TRUE))

year_ALS_data= list.files(pattern="year*")

#### example of how the data look like

In [8]:
#par(mfrow=c(2,2), mar=c(4,4,4,4))
#plot(stack(AGB_data[1])[[1]], main="AGB (t/ha)")
#plot(stack(AGB_data[1])[[2]], main="lower CI (t/ha)")
#plot(stack(AGB_data[1])[[3]], main="upper CI (t/ha)")

#plot(raster(year_ALS_data[1]), main="year ALS was collected")

## Load shapefile of areas of interest

In [9]:
aois=readOGR(getwd(),"aois")
aois_WGS84=spTransform(aois, "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ") # reporject to WGS 84 for plotting in leaflet

ERROR: Error in ogrInfo(dsn = dsn, layer = layer, encoding = encoding, use_iconv = use_iconv, : Cannot open data source


## Plot the aois using leaflet

In [6]:
# plot the aois using leaflet
#m <- leaflet(aois_WGS84) %>%
#    addTiles() %>% # Add default OpenStreetMap map tiles
#    addPolygons(color = "#444444", weight = 1, smoothFactor = 0.5,
#    opacity = 1.0, fillOpacity = 0.5, label = ~id,
#                labelOptions = labelOptions(noHide = T, textsize = "15px"))# %>%
#m  # Print the map

## Load ICESAT2 biomass tiles

In [5]:
# old tiles that Tony provided originally
#path_biomass_tiles="/projects/shared-buckets/tonyumd/Norway_agb_tiles"

# new tiles
#path_biomass_tiles=paste0("/projects/shared-buckets/lduncanson/norway_test_data/", dir_name)

path_biomass_tiles=paste0("/projects/my-private-bucket/dps_output/run_boreal_biomass_v2_ubuntu/master/2022/", dir_name)


# recursively find the tiles in the directory
tiles=list.files(path_biomass_tiles, full=T, pattern="*cog.tif", recursive=T)
length(tiles)

#if cog output not working
tiles= list.files(path=path_biomass_tiles, pattern="*tmp.tif", full=TRUE, recursive=TRUE)
#print(AGB_data)

tiles <- tiles[which(endsWith(tiles, 'tif'))]
print(tiles)

[1] 10

### create tile index (only if it's not already existing)

In [6]:
tile_index=gdalUtils::gdaltindex(
  paste0(output_dir_tiles,"prediction_tiles_",dir_name,".shp"),
  tiles
)
tile_index=readOGR(output_dir_tiles, paste0("prediction_tiles_",dir_name))

OGR data source with driver: ESRI Shapefile 
Source: "/projects/my-private-bucket/norway_test_data/tiles", layer: "prediction_tiles_baseline_run"
with 10 features
It has 1 fields


In [9]:
# reporject aois to match crs of ICESat-2 tiles
aois_reproject= spTransform(aois, raster::crs(tile_index))

In [10]:
# if one wants to check whether the ICESAT-2 biomass tiles cover all aois
#plot(aois_reproject)
#plot(tile_index, add=T, border="red")

# Now iterate through each SR16 AGB area and validate ICESAT2 biomass

### create output dir where outputs are saved (only if one wants to print out maps and plots)


In [11]:
#dir.create(paste0(getwd(),"/output"), showWarnings=F)

### create output dataframe for reporting RMSE and BIAS values

In [12]:
results=data.frame(ID=NA
                  ,RMSE=NA
                  ,BIAS=NA
                  ,RMSE_rel=NA
                  ,BIAS_rel=NA)

### Iterate through each aoi and validate

In [ ]:
for (i in 1:6){#nrow(aois)
   print(i)
    # select one aoi
    aoi=aois_reproject[i,]
    results[i,]$ID=aoi$id

    # select the aoi from ICESAT 2 that overlaps
    aoi_ICESAT= tile_index[gIntersects(tile_index, aoi, byid=T)[1,],]

    # selected the AGB from lidar and from ICESAT2
    AGB_SR16 = rast(AGB_data[AGB_data_IDs==aoi$id])[[1]]
    
    # if there are no tiles with ICESAT 2 data then skip
    if (nrow(aoi_ICESAT)==0){next}

    # if there is only one tile of ICESAT 2 data
    if (nrow(aoi_ICESAT)==1){
        AGB_ICESAT= rast(aoi_ICESAT$location)
        AGB_ICESAT= terra::crop(AGB_ICESAT, aoi)
        
    } 

    # if there are multiple of ICESAT 2 data, then mosaic them and get an average value
     if (nrow(aoi_ICESAT)>1){
         
        AGB_ICESAT= rast(aoi_ICESAT[1,]$location)
        AGB_ICESAT= terra::crop(AGB_ICESAT, aoi)

        for (j in 2:nrow(aoi_ICESAT)){
            one_more= rast(aoi_ICESAT[1,]$location)
            one_more= terra::crop(one_more, aoi)

            AGB_ICESAT= terra::mosaic((AGB_ICESAT),(one_more), fun="mean" )
            #AGB_ICESAT= AGB_ICESAT %>% raster()
        }
        AGB_ICESAT= terra::crop(AGB_ICESAT, aoi)
    } 
    
    # resample AGB from lidar to the same pizel size and crs of ICESAT2
    AGB_SR16_resampled= terra::resample(AGB_SR16,AGB_ICESAT )

    # mask out non-forest pixels
    AGB_ICESAT_masked= terra::mask(AGB_ICESAT,AGB_SR16_resampled )
   
    # compute residuals between the two maps
    residuals= AGB_SR16_resampled-AGB_ICESAT_masked[[AGB_band]]
    
    # write out raster results (only if one wants the )
    #raster::writeRaster(AGB_SR16_resampled, file=paste0(getwd(),"/output/reference_aoi_",aoi$id,".tif"), format="GTiff")
    #raster::writeRaster(AGB_ICESAT_masked, file=paste0(getwd(),"/output/ICESAT_aoi_",aoi$id,".tif"), format="GTiff")
    #raster::writeRaster(AGB_SR16_resampled, file=paste0(getwd(),"/output/residuals_aoi_",aoi$id,".tif"), format="GTiff")

   # compute RMSE and BIAS
    RMSE=as.numeric(sqrt(global(residuals^2, fun="mean", na.rm=T)))
    BIAS= as.numeric(global(residuals, fun="mean", na.rm=T))

    # compute relative RMSE and BIAS as % of the mean
    RMSE_rel= RMSE/as.numeric(global(AGB_SR16_resampled, fun="mean", na.rm=T))*100
    BIAS_rel= BIAS/as.numeric(global(AGB_SR16_resampled, fun="mean", na.rm=T))*100

    results[i,]$RMSE=RMSE
    results[i,]$BIAS=BIAS
    results[i,]$RMSE_rel=RMSE_rel
    results[i,]$BIAS_rel=BIAS_rel
    
    # 2d histogram with default option
    #scatterPlot= ggplot(data.frame(pred=terra::values(AGB_ICESAT_masked[[AGB_band]]), reference=terra::values(AGB_SR16_resampled))
     #      , aes(x=pred, y=reference) ) +
      #geom_bin2d(bins=70) +
      #scale_fill_continuous(type = "viridis") +
      #geom_abline(intercept = 0, slope = 1)+
      #ggtitle(paste0("aoi_n", aoi$id))
      #theme_bw()
   # export scatterplot
    #jpeg(paste0(getwd(),"/output/scatterplot_aoi_", aoi$id,".jpg"), width = 500, height = 500)
    #scatterPlot
    #dev.off()

}

print("run OK!")

In [ ]:
results=results[order(results$ID, decreasing=F),]
results %>% 
 mutate_if(is.numeric, round,2)

## plot again the aois to check where things aren't great

In [ ]:
# plot the aois using leaflet
m <- leaflet(aois_WGS84) %>%
    addTiles() %>% # Add default OpenStreetMap map tiles
    addPolygons(color = "#444444", weight = 1, smoothFactor = 0.5,
    opacity = 1.0, fillOpacity = 0.5, label = ~id,
                labelOptions = labelOptions(noHide = T, textsize = "15px"))# %>%
m  # Print the map

## Output to CSV

In [ ]:
csv_dir <- '/projects/my-public-bucket/norway_test_data/iteration_results/'
out_name <- paste(csv_dir, dir_name, '.csv', sep='')
print(out_name)
write.csv(results, file=out_name)